<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/Collagen-Pradeep/Bioco_Collagen_Klassiske_Superveiledet_Regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importering, lasting og formattering

Lasting av nødvendige bibliotek og pakker

In [4]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.0 MB/s eta 0:00:00


In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Importering av nødvendig bibliotek og pakker

In [6]:
import pandas as pd
import numpy as np
import copy
import math

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
import sklearn.model_selection
from statistics import mean

from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR


import optuna
from optuna.visualization import plot_optimization_history

# Utelukker utskriving av logg for hyperoptimalisering
# optuna.logging.set_verbosity(optuna.logging.WARNING)

Evalueringsmetrikker

In [7]:
from sklearn.metrics import (mean_absolute_error,
                             mean_squared_error,
                             mean_absolute_percentage_error,
                             r2_score)

# Lager en funksjon for RMSE
def rmse(y_faktisk, y_predikert):
  return np.sqrt(mean_squared_error(y_faktisk, y_predikert))

# Egendefinerte moduler

In [8]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/oppdeling_trening_testsett.py /content/oppdeling_trening_testsett.py

/content


In [9]:
from oppdeling_trening_testsett import trening_testsett_oppdeling_enzym
from oppdeling_trening_testsett import trening_testsett_oppdeling_dag
from oppdeling_trening_testsett import trening_validering_oppdeling_dag

In [10]:
def evaluering_beregning(y_test, y_test_prediksjon, y_trening, y_trening_prediksjon):
  rmse_test = rmse(y_test, y_test_prediksjon)
  r2_test = r2_score(y_test, y_test_prediksjon)
  mae_test = mean_absolute_error(y_test, y_test_prediksjon)
  mape_test = mean_absolute_percentage_error(y_test, y_test_prediksjon)

  rmse_trening = rmse(y_trening, y_trening_prediksjon)
  r2_trening = r2_score(y_trening, y_trening_prediksjon)
  mae_trening = mean_absolute_error(y_trening, y_trening_prediksjon)
  mape_trening = mean_absolute_percentage_error(y_trening, y_trening_prediksjon)

  return rmse_test, r2_test, mae_test, mape_test, rmse_trening, r2_trening, mae_trening, mape_trening

In [11]:
def evaluering_resultater(detailed_objective):
  rmse_test = detailed_objective(study.best_trial)[0]
  r2_test = detailed_objective(study.best_trial)[1]
  mae_test = detailed_objective(study.best_trial)[2]
  mape_test = detailed_objective(study.best_trial)[3]

  rmse_trening = detailed_objective(study.best_trial)[4]
  r2_trening = detailed_objective(study.best_trial)[5]
  mae_trening = detailed_objective(study.best_trial)[6]
  mape_trening = detailed_objective(study.best_trial)[7]

  return rmse_test, r2_test, mae_test, mape_test, rmse_trening, r2_trening, mae_trening, mape_trening

In [12]:
def rmse_validering_StratifiedKFold(modell, datasett, respons, n_splitt):

  skf = StratifiedKFold(n_splits=n_splitt)

  rmse_validering_resultat = []

  for trening, validering in skf.split(datasett, respons):
    X_trening_fold = (datasett.iloc[trening, :]).iloc[:, :-1]
    X_validering_fold = (datasett.iloc[validering, :]).iloc[:, :-1]

    y_trening_fold = (datasett.iloc[trening, :]).iloc[:, -1]
    y_validering_fold = (datasett.iloc[validering, :]).iloc[:, -1]

    modell.fit(X_trening_fold, y_trening_fold)

    y_validering_prediksjon = modell.predict(X_validering_fold)

    rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
    rmse_validering_resultat.append(rmse_validering)

  return mean(rmse_validering)

# Importering av relevant data

In [13]:
# Velger første kolonne med dato og tid som index
collagen_data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data.csv',
                            header=0,
                            sep=',',
                            index_col=0)

# Normal produksjon
collagen_data_normal = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data_normal.csv',
                                   header=0,
                                   sep=',',
                                   index_col=0)

collagen_data_design = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data_design.csv',
                                   header=0,
                                   sep=',',
                                   index_col=0)

# Importerer rådata
rå_data = pd.read_csv('/content/drive/MyDrive/MasterV24/BiocoData.csv',
                      header=0,
                      sep=';',
                      index_col=0)

# Formatterer index til riktig format og datatype
collagen_data.index = pd.to_datetime(collagen_data.index,
                                     format='%Y-%m-%d %H:%M:%S')

collagen_data_normal.index = pd.to_datetime(collagen_data_normal.index,
                                            format='%Y-%m-%d %H:%M:%S')

collagen_data_design.index = pd.to_datetime(collagen_data_design.index,
                                            format='%Y-%m-%d %H:%M:%S')

rå_data.index = pd.to_datetime(rå_data.index,
                               format='%d-%m-%Y %H:%M:%S.%f')

Tilfeldighetsfrø

In [14]:
#Tilfeldighetsfrø
random_seed = 123

# Beskrivelse av datasett

In [15]:
collagen_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29136 entries, 2022-10-31 17:37:00 to 2023-06-14 01:06:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  29136 non-null  int64  
 1   EnzymeType_A2  29136 non-null  int64  
 2   EnzymeType_B   29136 non-null  int64  
 3   EnzymeType_C   29136 non-null  int64  
 4   EnzymeType_D   29136 non-null  int64  
 5   EnzymeType_E   29136 non-null  int64  
 6   RawMatPercent  29136 non-null  float64
 7   NIRfat         29136 non-null  float64
 8   NIRash         29136 non-null  float64
 9   NIRwater       29136 non-null  float64
 10  TT08           29136 non-null  float64
 11  TT20           29136 non-null  float64
 12  TT12           29136 non-null  float64
 13  Collagen       89 non-null     float64
dtypes: float64(8), int64(6)
memory usage: 3.3 MB


In [16]:
collagen_data_normal.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 18568 entries, 2022-11-02 00:00:00 to 2023-06-14 01:06:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  18568 non-null  int64  
 1   EnzymeType_A2  18568 non-null  int64  
 2   EnzymeType_B   18568 non-null  int64  
 3   EnzymeType_C   18568 non-null  int64  
 4   EnzymeType_D   18568 non-null  int64  
 5   EnzymeType_E   18568 non-null  int64  
 6   RawMatPercent  18568 non-null  float64
 7   NIRfat         18568 non-null  float64
 8   NIRash         18568 non-null  float64
 9   NIRwater       18568 non-null  float64
 10  TT08           18568 non-null  float64
 11  TT20           18568 non-null  float64
 12  TT12           18568 non-null  float64
 13  Collagen       31 non-null     float64
dtypes: float64(8), int64(6)
memory usage: 2.1 MB


In [17]:
collagen_data_design.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10568 entries, 2022-10-31 17:37:00 to 2023-06-13 23:59:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  10568 non-null  int64  
 1   EnzymeType_A2  10568 non-null  int64  
 2   EnzymeType_B   10568 non-null  int64  
 3   EnzymeType_C   10568 non-null  int64  
 4   EnzymeType_D   10568 non-null  int64  
 5   EnzymeType_E   10568 non-null  int64  
 6   RawMatPercent  10568 non-null  float64
 7   NIRfat         10568 non-null  float64
 8   NIRash         10568 non-null  float64
 9   NIRwater       10568 non-null  float64
 10  TT08           10568 non-null  float64
 11  TT20           10568 non-null  float64
 12  TT12           10568 non-null  float64
 13  Collagen       58 non-null     float64
dtypes: float64(8), int64(6)
memory usage: 1.2 MB


# Moduler for optimalisering etter treningsandel etter algoritme

## Modelltrening etter enzymtype

### RandomForestRegressor

In [18]:
def trening_rfr_enzym(treningssett_markert,
                      enzymtyper_treningssett_markert,
                      treningssett_umarkert,
                      treningssett,
                      X_test,
                      y_test,
                      splitt,
                      gjentagelser,
                      n_runder,
                      trenings_andel = 1.0
                      ):

    X_trening = treningssett.iloc[:, :-1]
    X_trening_markert = treningssett_markert.iloc[:, :-1]
    X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
    y_trening_markert = treningssett_markert.iloc[:, -1]

    # RandomForestRegressor
    def objective(trial):

      parametere = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 200),
            'max_depth': trial.suggest_int('max_depth', 1, 20)
        }
      rfr_pipeline = Pipeline([
            ('skalerer', StandardScaler()),
            ('modell', RandomForestRegressor(**parametere,
                                            random_state=random_seed))
        ])

      rskf = RepeatedStratifiedKFold(n_splits = splitt,
                                    n_repeats=gjentagelser,
                                    random_state=random_seed)

      rmse_validering_resultat = []

      for train, test in rskf.split(treningssett_markert,
                                    enzymtyper_treningssett_markert):

        X_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, :-1]
        X_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, :-1]

        y_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, -1]
        y_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, -1]

        rfr_pipeline.fit(X_trening_fold, y_trening_fold)

        y_validering_prediksjon = rfr_pipeline.predict(X_validering_fold)
        rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
        rmse_validering_resultat.append(rmse_validering)

      rmse_validering_snitt = mean(rmse_validering_resultat)

      return rmse_validering_snitt

    if __name__ == "__main__":
      study = optuna.create_study(direction='minimize')
      study.optimize(objective, n_trials=n_runder)


    # Bruker beste paramtere for studiet for å lage modell
    beste_parametere = study.best_params

    rfr_pipeline = Pipeline([
        ('skalerer', StandardScaler()),
        ('modell', RandomForestRegressor(**beste_parametere,
                                        random_state=random_seed))
    ])

    # Trener modell for videre evaluering
    rfr_pipeline.fit(X_trening_markert, y_trening_markert)
    y_test_prediksjon = rfr_pipeline.predict(X_test)
    y_trening_prediksjon = rfr_pipeline.predict(X_trening_markert)

    # Henter ut evalueringsresulateter av beste modell fra undersøkelse
    rmse_test, r2_test, mae_test, mape_test,\
    rmse_trening, r2_trening, mae_trening, mape_trening\
    = evaluering_beregning(y_test,
                          y_test_prediksjon,
                          y_trening_markert,
                          y_trening_prediksjon)


    # Legger resulater til i datasett
    resultater = [trenings_andel,
                  rmse_test, r2_test, mae_test, mape_test,
                  rmse_trening, r2_trening, mae_trening, mape_trening,
                  beste_parametere]

    return resultater

### KNeighborsRegressor

In [19]:
def trening_knr_enzym(treningssett_markert,
                      enzymtyper_treningssett_markert,
                      treningssett_umarkert,
                      treningssett,
                      X_test,
                      y_test,
                      splitt,
                      gjentagelser,
                      n_runder,
                      trenings_andel=1.0):


    X_trening = treningssett.iloc[:, :-1]
    X_trening_markert = treningssett_markert.iloc[:, :-1]
    X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
    y_trening_markert = treningssett_markert.iloc[:, -1]

    # KNR
    def objective(trial):

      parametere = {
            'n_neighbors': trial.suggest_int('n_neighbors', 1, 10),
            'p': trial.suggest_int('p', 1, 10)
        }
      knr_pipeline = Pipeline([
            ('skalerer', StandardScaler()),
            ('modell', KNeighborsRegressor(**parametere))
        ])

      rskf = RepeatedStratifiedKFold(n_splits = splitt,
                                    n_repeats=gjentagelser,
                                    random_state=random_seed)

      rmse_validering_resultat = []

      for train, test in rskf.split(treningssett_markert,
                                    enzymtyper_treningssett_markert):

        X_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, :-1]
        X_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, :-1]

        y_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, -1]
        y_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, -1]

        knr_pipeline.fit(X_trening_fold, y_trening_fold)

        y_validering_prediksjon = knr_pipeline.predict(X_validering_fold)

        rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
        rmse_validering_resultat.append(rmse_validering)

      rmse_validering_snitt = mean(rmse_validering_resultat)

      return rmse_validering_snitt

    if __name__ == "__main__":
      study = optuna.create_study(direction='minimize')
      study.optimize(objective, n_trials=n_runder)

    # Bruker beste paramtere for studiet for å lage modell
    beste_parametere = study.best_params

    knr_pipeline = Pipeline([
            ('skalerer', StandardScaler()),
            ('modell', KNeighborsRegressor(**beste_parametere))
        ])

    # Trener modell for videre evaluering
    knr_pipeline.fit(X_trening_markert, y_trening_markert)
    y_test_prediksjon = knr_pipeline.predict(X_test)
    y_trening_prediksjon = knr_pipeline.predict(X_trening_markert)

    # Henter ut evalueringsresulateter av beste modell fra undersøkelse
    rmse_test, r2_test,\
    mae_test, mape_test,\
    rmse_trening, r2_trening,\
    mae_trening, mape_trening = evaluering_beregning(y_test,
                                                    y_test_prediksjon,
                                                    y_trening_markert,
                                                    y_trening_prediksjon)

    # Legger resulater til i datasett
    resultater = [trenings_andel,
                  rmse_test, r2_test, mae_test, mape_test,
                  rmse_trening, r2_trening, mae_trening, mape_trening,
                  beste_parametere]

    return resultater

### Support Vector Regressor

In [20]:
def trening_svr_enzym(treningssett_markert,
                      enzymtyper_treningssett_markert,
                      treningssett_umarkert,
                      treningssett,
                      X_test,
                      y_test,
                      splitt,
                      gjentagelser,
                      n_runder,
                      trenings_andel=1.0):


    X_trening = treningssett.iloc[:, :-1]
    X_trening_markert = treningssett_markert.iloc[:, :-1]
    X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
    y_trening_markert = treningssett_markert.iloc[:, -1]

    # SVR
    def objective(trial):

      parametere = {
            'kernel': trial.suggest_categorical('kernel',["linear", "poly", "rbf"]),
            'C': trial.suggest_int('C', 1, 100),
            'degree': trial.suggest_int('degree', 1, 20),
            'gamma': trial.suggest_categorical('gamma', ['scale', 'auto'])

        }

      svr_pipeline = Pipeline([
            ('skalerer', StandardScaler()),
            ('modell', SVR(**parametere))
        ])

      rskf = RepeatedStratifiedKFold(n_splits = splitt,
                                    n_repeats=gjentagelser,
                                    random_state=random_seed)

      rmse_validering_resultat = []

      for train, test in rskf.split(treningssett_markert,
                                    enzymtyper_treningssett_markert):

        X_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, :-1]
        X_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, :-1]

        y_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, -1]
        y_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, -1]

        svr_pipeline.fit(X_trening_fold, y_trening_fold)

        y_validering_prediksjon = svr_pipeline.predict(X_validering_fold)

        rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
        rmse_validering_resultat.append(rmse_validering)

      rmse_validering_snitt = mean(rmse_validering_resultat)

      return rmse_validering_snitt

    if __name__ == "__main__":
      study = optuna.create_study(direction='minimize')
      study.optimize(objective, n_trials=n_runder)

    # Bruker beste paramtere for studiet for å lage modell
    beste_parametere = study.best_params

    svr_pipeline = Pipeline([
            ('skalerer', StandardScaler()),
            ('modell', SVR(**beste_parametere))
        ])

    # Trener modell for videre evaluering
    svr_pipeline.fit(X_trening_markert, y_trening_markert)
    y_test_prediksjon = svr_pipeline.predict(X_test)
    y_trening_prediksjon = svr_pipeline.predict(X_trening_markert)

    # Henter ut evalueringsresulateter av beste modell fra undersøkelse
    rmse_test, r2_test,\
    mae_test, mape_test,\
    rmse_trening, r2_trening,\
    mae_trening, mape_trening = evaluering_beregning(y_test,
                                                    y_test_prediksjon,
                                                    y_trening_markert,
                                                    y_trening_prediksjon)

    # Legger resulater til i datasett
    resultater = [trenings_andel,
                  rmse_test, r2_test, mae_test, mape_test,
                  rmse_trening, r2_trening, mae_trening, mape_trening,
                  beste_parametere]

    return resultater

## Modelltrening etter dag og kontinuitet

### RandomForestRegressor

In [21]:
def trening_rfr_dag(trenings_andel,
                    treningssett_markert,
                    treningssett_umarkert,
                    treningssett,
                    X_test,
                    y_test,
                    splitt,
                    n_runder):

  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # RandomForestRegressor
  def objective(trial):

    parametere = {
          'n_estimators': trial.suggest_int('n_estimators', 100, 200),
          'max_depth': trial.suggest_int('max_depth', 1, 20)
      }
    rfr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', RandomForestRegressor(**parametere,
                                           random_state=random_seed))
      ])

    treningssett, valideringssett = trening_validering_oppdeling_dag(rå_data,
                                                                     treningssett_markert,
                                                                     n_splitt=splitt,
                                                                     trening_krav=True)
    rmse_validering_resultat = []

    for i, j in zip(treningssett, valideringssett):
      train = treningssett[i]
      val = valideringssett[j]

      X_trening_fold = (trening_markert.loc[train, :]).iloc[:, :-1]
      X_validering_fold = (trening_markert.loc[val, :]).iloc[:, :-1]

      y_trening_fold = (trening_markert.loc[train, :]).iloc[:, -1]
      y_validering_fold = (trening_markert.loc[val, :]).iloc[:, -1]

      rfr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = rfr_pipeline.predict(X_validering_fold)
      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt


  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_runder)


  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  rfr_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**beste_parametere,
                                       random_state=random_seed))
  ])

  # Trener modell for videre evaluering
  rfr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = rfr_pipeline.predict(X_test)
  y_trening_prediksjon = rfr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test, mae_test, mape_test,\
  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_beregning(y_test,
                         y_test_prediksjon,
                         y_trening_markert,
                         y_trening_prediksjon)


  # Legger resulater til i datasett
  resultater = [trenings_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  return resultater

### KNeighborsRegressor

In [22]:
def trening_knr_dag(trenings_andel,
                    treningssett_markert,
                    treningssett_umarkert,
                    treningssett,
                    X_test,
                    y_test,
                    splitt,
                    n_runder):

    X_trening = treningssett.iloc[:, :-1]
    X_trening_markert = treningssett_markert.iloc[:, :-1]
    X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
    y_trening_markert = treningssett_markert.iloc[:, -1]

    # KneigborsRegressor
    def objective(trial):

      parametere = {
            'n_neighbors': trial.suggest_int('n_neighbors', 1, 20),
            'p': trial.suggest_int('p', 1, 20)
        }
      knr_pipeline = Pipeline([
            ('skalerer', StandardScaler()),
            ('modell', KNeighborsRegressor(**parametere))
        ])


      treningssett, valideringssett = trening_validering_oppdeling_dag(rå_data,
                                                                        treningssett_markert,
                                                                        n_splitt=splitt,
                                                                        trening_krav=True)
      rmse_validering_resultat = []

      for i, j in zip(treningssett, valideringssett):
        train = treningssett[i]
        val = valideringssett[j]

        X_trening_fold = (trening_markert.loc[train, :]).iloc[:, :-1]
        X_validering_fold = (trening_markert.loc[val, :]).iloc[:, :-1]

        y_trening_fold = (trening_markert.loc[train, :]).iloc[:, -1]
        y_validering_fold = (trening_markert.loc[val, :]).iloc[:, -1]

        knr_pipeline.fit(X_trening_fold, y_trening_fold)

        y_validering_prediksjon = knr_pipeline.predict(X_validering_fold)
        rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
        rmse_validering_resultat.append(rmse_validering)

      rmse_validering_snitt = mean(rmse_validering_resultat)

      return rmse_validering_snitt


    if __name__ == "__main__":
      study = optuna.create_study(direction='minimize')
      study.optimize(objective, n_trials=n_runder)


    # Bruker beste paramtere for studiet for å lage modell
    beste_parametere = study.best_params

    knr_pipeline = Pipeline([
            ('skalerer', StandardScaler()),
            ('modell', KNeighborsRegressor(**beste_parametere))
        ])

    # Trener modell for videre evaluering
    knr_pipeline.fit(X_trening_markert, y_trening_markert)
    y_test_prediksjon = knr_pipeline.predict(X_test)
    y_trening_prediksjon = knr_pipeline.predict(X_trening_markert)

    # Henter ut evalueringsresulateter av beste modell fra undersøkelse
    rmse_test, r2_test,\
    mae_test, mape_test,\
    rmse_trening, r2_trening,\
    mae_trening, mape_trening = evaluering_beregning(y_test,
                                                    y_test_prediksjon,
                                                    y_trening_markert,
                                                    y_trening_prediksjon)

    # Legger resulater til i datasett
    resultater = [trenings_andel,
                  rmse_test, r2_test, mae_test, mape_test,
                  rmse_trening, r2_trening, mae_trening, mape_trening,
                  beste_parametere]


    return resultater

### Support Vector Regressor

In [23]:
def trening_svr_dag(trenings_andel,
                    treningssett_markert,
                    treningssett_umarkert,
                    treningssett,
                    X_test,
                    y_test,
                    splitt,
                    n_runder):


    X_trening = treningssett.iloc[:, :-1]
    X_trening_markert = treningssett_markert.iloc[:, :-1]
    X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
    y_trening_markert = treningssett_markert.iloc[:, -1]

    # SupportVectorRegressor
    def objective(trial):

      parametere = {
            'kernel': trial.suggest_categorical('kernel',["linear", "poly", "rbf"]),
            'C': trial.suggest_int('C', 1, 100),
            'degree': trial.suggest_int('degree', 1, 20),
            'gamma': trial.suggest_categorical('gamma', ['scale', 'auto'])

        }

      svr_pipeline = Pipeline([
            ('skalerer', StandardScaler()),
            ('modell', SVR(**parametere))
        ])


      treningssett, valideringssett = trening_validering_oppdeling_dag(rå_data,
                                                                        treningssett_markert,
                                                                        n_splitt=splitt,
                                                                        trening_krav=True)
      rmse_validering_resultat = []

      for i, j in zip(treningssett, valideringssett):
        train = treningssett[i]
        val = valideringssett[j]

        X_trening_fold = (trening_markert.loc[train, :]).iloc[:, :-1]
        X_validering_fold = (trening_markert.loc[val, :]).iloc[:, :-1]

        y_trening_fold = (trening_markert.loc[train, :]).iloc[:, -1]
        y_validering_fold = (trening_markert.loc[val, :]).iloc[:, -1]

        svr_pipeline.fit(X_trening_fold, y_trening_fold)

        y_validering_prediksjon = svr_pipeline.predict(X_validering_fold)
        rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
        rmse_validering_resultat.append(rmse_validering)

      rmse_validering_snitt = mean(rmse_validering_resultat)

      return rmse_validering_snitt


    if __name__ == "__main__":
      study = optuna.create_study(direction='minimize')
      study.optimize(objective, n_trials=n_runder)


    # Bruker beste paramtere for studiet for å lage modell
    beste_parametere = study.best_params

    svr_pipeline = Pipeline([
            ('skalerer', StandardScaler()),
            ('modell', SVR(**beste_parametere))
        ])

    # Trener modell for videre evaluering
    svr_pipeline.fit(X_trening_markert, y_trening_markert)
    y_test_prediksjon = svr_pipeline.predict(X_test)
    y_trening_prediksjon = svr_pipeline.predict(X_trening_markert)

    # Henter ut evalueringsresulateter av beste modell fra undersøkelse
    rmse_test, r2_test,\
    mae_test, mape_test,\
    rmse_trening, r2_trening,\
    mae_trening, mape_trening = evaluering_beregning(y_test,
                                                    y_test_prediksjon,
                                                    y_trening_markert,
                                                    y_trening_prediksjon)

    # Legger resulater til i datasett
    resultater = [trenings_andel,
                  rmse_test, r2_test, mae_test, mape_test,
                  rmse_trening, r2_trening, mae_trening, mape_trening,
                  beste_parametere]

    return resultater

## Lagring av resultater

In [24]:
# Kolonner til dataframe for lagring av resultater fra opmtimalisering
resultater_kolonner = ["Trenings andel",
                       "RMSE test",
                       "R2 test",
                       "MAE test",
                       "MAPE test",
                       "RMSE trening",
                       "R2 trening",
                       "MAE trening",
                       "MAPE trening",
                       "Beste parametere"]


# NIR målinger inkludert

## Hele datasett

## Alternativ 1: Et testsett. Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

In [22]:
# Dataframes for lagring av resultater
rfr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)

# Definerer trening og testandel
trenings_andel = 1.0
test_andel = 0.2


# Deler datasettet i trening og testsett
treningssett_markert, enzymtyper_treningssett_markert, testsett, _,\
treningssett_umarkert, _, treningssett\
=trening_testsett_oppdeling_enzym(rå_data,
                                  collagen_data,
                                  test_andel=test_andel)


# Treningssett i forklaringsvariabler og respons
X_test = testsett.iloc[:, :-1]
y_test = testsett.iloc[:, -1]

# Kryssvalidering
splitt = 2
gjentagelser = 10

# Antall runder til optimalisering
n_runder = 300

### RandomForestRegressor

In [23]:
# Modelloptimalisering
rfr_resultater = trening_rfr_enzym(treningssett_markert,
                                    enzymtyper_treningssett_markert,
                                    treningssett_umarkert,
                                    treningssett,
                                    X_test,
                                    y_test,
                                    splitt = splitt,
                                    gjentagelser = gjentagelser,
                                    n_runder = n_runder,
                                    trenings_andel=trenings_andel)

# Lagring av resultater
rfr_resultater_enzym.loc[len(rfr_resultater_enzym)] = rfr_resultater

# Data med resultater fra treningsandeler
rfr_resultater_enzym

[I 2024-03-16 16:18:05,372] A new study created in memory with name: no-name-d7e7cac3-b7ba-42e2-bdec-db21ed8b5c80
[I 2024-03-16 16:18:18,460] Trial 0 finished with value: 4.09848229803199 and parameters: {'n_estimators': 189, 'max_depth': 1}. Best is trial 0 with value: 4.09848229803199.
[I 2024-03-16 16:18:22,552] Trial 1 finished with value: 3.6888234117483645 and parameters: {'n_estimators': 136, 'max_depth': 5}. Best is trial 1 with value: 3.6888234117483645.
[I 2024-03-16 16:18:28,743] Trial 2 finished with value: 3.67985935584939 and parameters: {'n_estimators': 178, 'max_depth': 4}. Best is trial 2 with value: 3.67985935584939.
[I 2024-03-16 16:18:33,792] Trial 3 finished with value: 3.6713174483631668 and parameters: {'n_estimators': 187, 'max_depth': 14}. Best is trial 3 with value: 3.6713174483631668.
[I 2024-03-16 16:18:38,103] Trial 4 finished with value: 3.69733123055526 and parameters: {'n_estimators': 169, 'max_depth': 3}. Best is trial 3 with value: 3.6713174483631668.


,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,1.0,3.982475,0.543615,3.455185,0.175218,1.215717,0.941674,0.933375,0.045947,"{'n_estimators': 191, 'max_depth': 10}"


### KNeighborsRegressor

In [24]:
# Modelloptimalisering
knr_resultater = trening_knr_enzym(treningssett_markert,
                                    enzymtyper_treningssett_markert,
                                    treningssett_umarkert,
                                    treningssett,
                                    X_test,
                                    y_test,
                                    splitt=splitt,
                                    gjentagelser=gjentagelser,
                                    n_runder=n_runder,
                                    trenings_andel=trenings_andel)


# Lagring av resultater
knr_resultater_enzym.loc[len(knr_resultater_enzym)] = knr_resultater

# Data med resultater fra treningsandeler
knr_resultater_enzym

[I 2024-03-16 16:45:57,778] A new study created in memory with name: no-name-99ae79b1-c5a0-4f65-96c2-5231beb5e373
[I 2024-03-16 16:45:57,940] Trial 0 finished with value: 4.675899004883179 and parameters: {'n_neighbors': 2, 'p': 3}. Best is trial 0 with value: 4.675899004883179.
[I 2024-03-16 16:45:58,096] Trial 1 finished with value: 4.34428459634309 and parameters: {'n_neighbors': 8, 'p': 9}. Best is trial 1 with value: 4.34428459634309.
[I 2024-03-16 16:45:58,267] Trial 2 finished with value: 4.320375914198185 and parameters: {'n_neighbors': 8, 'p': 3}. Best is trial 2 with value: 4.320375914198185.
[I 2024-03-16 16:45:58,437] Trial 3 finished with value: 4.42396712916974 and parameters: {'n_neighbors': 5, 'p': 5}. Best is trial 2 with value: 4.320375914198185.
[I 2024-03-16 16:45:58,603] Trial 4 finished with value: 4.540735472061479 and parameters: {'n_neighbors': 3, 'p': 6}. Best is trial 2 with value: 4.320375914198185.
[I 2024-03-16 16:45:58,781] Trial 5 finished with value: 4.

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,1.0,4.220846,0.487346,3.451278,0.18421,3.754695,0.443651,3.135042,0.148496,"{'n_neighbors': 10, 'p': 2}"


### SupportVectorRegressor

In [25]:
# Modelloptimalisering
svr_resultater = trening_svr_enzym(treningssett_markert,
                                    enzymtyper_treningssett_markert,
                                    treningssett_umarkert,
                                    treningssett,
                                    X_test,
                                    y_test,
                                    splitt=splitt,
                                    gjentagelser=gjentagelser,
                                    n_runder=n_runder,
                                    trenings_andel=trenings_andel)


# Lagring av resultater
svr_resultater_enzym.loc[len(svr_resultater_enzym)] = svr_resultater

# Data med resultater fra treningsandeler
svr_resultater_enzym

[I 2024-03-16 16:46:55,239] A new study created in memory with name: no-name-b2fb4a9c-cc9e-4310-844b-9524789df0d6
[I 2024-03-16 16:46:55,392] Trial 0 finished with value: 969.3638106224172 and parameters: {'kernel': 'poly', 'C': 52, 'degree': 3, 'gamma': 'scale'}. Best is trial 0 with value: 969.3638106224172.
[I 2024-03-16 16:46:55,955] Trial 1 finished with value: 7.072536027384748 and parameters: {'kernel': 'linear', 'C': 66, 'degree': 13, 'gamma': 'auto'}. Best is trial 1 with value: 7.072536027384748.
[I 2024-03-16 16:46:56,633] Trial 2 finished with value: 7.072285353611755 and parameters: {'kernel': 'linear', 'C': 90, 'degree': 6, 'gamma': 'scale'}. Best is trial 2 with value: 7.072285353611755.
[I 2024-03-16 16:46:56,952] Trial 3 finished with value: 7.048160756845111 and parameters: {'kernel': 'linear', 'C': 11, 'degree': 15, 'gamma': 'auto'}. Best is trial 3 with value: 7.048160756845111.
[I 2024-03-16 16:46:57,212] Trial 4 finished with value: 4.305595571625739 and parameter

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,1.0,4.077067,0.521677,3.468332,0.176324,2.591188,0.735031,1.71972,0.088713,"{'kernel': 'rbf', 'C': 5, 'degree': 20, 'gamma..."


## Alternativ 1: Flere treningsandeler

In [26]:
# Dataframes for lagring av resultater
rfr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)


# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, _, trening_umarkert,\
_, trening = trening_testsett_oppdeling_enzym(rå_data,
                                              collagen_data,
                                              test_andel=test_andel)

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [0.9, 1.0]

# Parametervalg for kryssvalidering
splitt = 2
gjentagelser = 10

# Antall runder til optimalisering
n_runder = 300

### RandomForestRegressor

In [27]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening


  # Modelloptimalisering
  rfr_resultater = trening_rfr_enzym(treningssett_markert,
                                     enzymtyper_treningssett_markert,
                                     treningssett_umarkert,
                                     treningssett,
                                     X_test,
                                     y_test,
                                     splitt = splitt,
                                     gjentagelser = gjentagelser,
                                     n_runder = n_runder,
                                     trenings_andel=trenings_andel)

  # Lagring av resultater
  rfr_resultater_enzym.loc[len(rfr_resultater_enzym)] = rfr_resultater

# Data med resultater fra treningsandeler
rfr_resultater_enzym

[I 2024-03-16 16:47:56,525] A new study created in memory with name: no-name-ad30c6ce-b838-446b-9d82-2d62762c94ee
[I 2024-03-16 16:47:59,485] Trial 0 finished with value: 3.4756526639891265 and parameters: {'n_estimators': 108, 'max_depth': 19}. Best is trial 0 with value: 3.4756526639891265.
[I 2024-03-16 16:48:04,370] Trial 1 finished with value: 3.464819125721198 and parameters: {'n_estimators': 173, 'max_depth': 6}. Best is trial 1 with value: 3.464819125721198.
[I 2024-03-16 16:48:10,806] Trial 2 finished with value: 3.480381559550004 and parameters: {'n_estimators': 159, 'max_depth': 10}. Best is trial 1 with value: 3.464819125721198.
[I 2024-03-16 16:48:16,081] Trial 3 finished with value: 3.476355968595721 and parameters: {'n_estimators': 192, 'max_depth': 18}. Best is trial 1 with value: 3.464819125721198.
[I 2024-03-16 16:48:20,968] Trial 4 finished with value: 3.4802657961578394 and parameters: {'n_estimators': 134, 'max_depth': 4}. Best is trial 1 with value: 3.464819125721

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.9,3.741385,0.597199,3.139492,0.161409,1.184373,0.944368,0.972105,0.046771,"{'n_estimators': 173, 'max_depth': 6}"
1,1.0,3.982475,0.543615,3.455185,0.175218,1.215717,0.941674,0.933375,0.045947,"{'n_estimators': 191, 'max_depth': 10}"


### KNeighborRegressor

In [28]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  knr_resultater = trening_knr_enzym(treningssett_markert,
                                     enzymtyper_treningssett_markert,
                                     treningssett_umarkert,
                                     treningssett,
                                     X_test,
                                     y_test,
                                     splitt=splitt,
                                     gjentagelser=gjentagelser,
                                     n_runder=n_runder,
                                     trenings_andel=trenings_andel)


  # Lagring av resultater
  knr_resultater_enzym.loc[len(knr_resultater_enzym)] = knr_resultater

# Data med resultater fra treningsandeler
knr_resultater_enzym

[I 2024-03-16 17:41:00,557] A new study created in memory with name: no-name-e1fffff1-f93b-47af-bd2c-dc6f1040006b
[I 2024-03-16 17:41:00,714] Trial 0 finished with value: 4.085144742166133 and parameters: {'n_neighbors': 8, 'p': 7}. Best is trial 0 with value: 4.085144742166133.
[I 2024-03-16 17:41:00,871] Trial 1 finished with value: 4.080409783106392 and parameters: {'n_neighbors': 4, 'p': 1}. Best is trial 1 with value: 4.080409783106392.
[I 2024-03-16 17:41:01,045] Trial 2 finished with value: 4.055368352622582 and parameters: {'n_neighbors': 7, 'p': 10}. Best is trial 2 with value: 4.055368352622582.
[I 2024-03-16 17:41:01,228] Trial 3 finished with value: 4.169632368266194 and parameters: {'n_neighbors': 3, 'p': 9}. Best is trial 2 with value: 4.055368352622582.
[I 2024-03-16 17:41:01,403] Trial 4 finished with value: 4.017207589829303 and parameters: {'n_neighbors': 5, 'p': 5}. Best is trial 4 with value: 4.017207589829303.
[I 2024-03-16 17:41:01,559] Trial 5 finished with value

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.9,4.526954,0.410291,3.757160,0.200553,3.634360,0.476153,3.063704,0.143876,"{'n_neighbors': 9, 'p': 2}"
1,1.0,4.220846,0.487346,3.451278,0.184210,3.754695,0.443651,3.135042,0.148496,"{'n_neighbors': 10, 'p': 2}"


### SupportVectorRegressor

In [29]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  svr_resultater = trening_svr_enzym(treningssett_markert,
                                     enzymtyper_treningssett_markert,
                                     treningssett_umarkert,
                                     treningssett,
                                     X_test,
                                     y_test,
                                     splitt=splitt,
                                     gjentagelser=gjentagelser,
                                     n_runder=n_runder,
                                     trenings_andel=trenings_andel)


  # Lagring av resultater
  svr_resultater_enzym.loc[len(svr_resultater_enzym)] = svr_resultater

# Data med resultater fra treningsandeler
svr_resultater_enzym

[I 2024-03-16 17:42:56,739] A new study created in memory with name: no-name-88365d02-ce41-401f-8c1b-a50067025f6d
[I 2024-03-16 17:42:56,889] Trial 0 finished with value: 4.281893335086923 and parameters: {'kernel': 'rbf', 'C': 96, 'degree': 8, 'gamma': 'auto'}. Best is trial 0 with value: 4.281893335086923.
[I 2024-03-16 17:42:57,040] Trial 1 finished with value: 31364786000.71914 and parameters: {'kernel': 'poly', 'C': 11, 'degree': 9, 'gamma': 'auto'}. Best is trial 0 with value: 4.281893335086923.
[I 2024-03-16 17:42:57,279] Trial 2 finished with value: 6.457709499196873 and parameters: {'kernel': 'linear', 'C': 31, 'degree': 15, 'gamma': 'scale'}. Best is trial 0 with value: 4.281893335086923.
[I 2024-03-16 17:42:57,434] Trial 3 finished with value: 4.116568297875073 and parameters: {'kernel': 'rbf', 'C': 22, 'degree': 10, 'gamma': 'auto'}. Best is trial 3 with value: 4.116568297875073.
[I 2024-03-16 17:42:57,594] Trial 4 finished with value: 198932008929695.25 and parameters: {'k

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.9,4.190605,0.494666,3.556132,0.182644,2.529868,0.746169,1.578403,0.082086,"{'kernel': 'rbf', 'C': 4, 'degree': 8, 'gamma'..."
1,1.0,4.077067,0.521677,3.468332,0.176324,2.591188,0.735031,1.719720,0.088713,"{'kernel': 'rbf', 'C': 5, 'degree': 11, 'gamma..."


## Alternativ 2: Et testsett fordelt på dag og kontinuitet

In [ ]:
# Dataframes for lagring av resultater
rfr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

# Definerer trening og testandel
trenings_andel = 1.0
test_andel = 0.2


# Deler datasettet i trening og testsett
trening_markert, test, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 collagen_data,
                                 test_andel=test_andel,
                                 trening_krav=True)



# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 300

### RandomForestregressor

In [31]:
# Modelloptimalisering
rfr_resultater = trening_rfr_dag(trenings_andel,
                                trening_markert,
                                trening_umarkert,
                                trening,
                                X_test,
                                y_test,
                                splitt = splitt,
                                n_runder = n_runder)

# Lagring av resultater
rfr_resultater_dag.loc[len(rfr_resultater_dag)] = rfr_resultater

# Data med resultater fra treningsandeler
rfr_resultater_dag

[I 2024-03-16 17:44:58,111] A new study created in memory with name: no-name-11abb8c1-2940-48f2-9c37-ba8af0e7ed0a
[I 2024-03-16 17:44:58,910] Trial 0 finished with value: 4.053731153563586 and parameters: {'n_estimators': 179, 'max_depth': 18}. Best is trial 0 with value: 4.053731153563586.
[I 2024-03-16 17:44:59,773] Trial 1 finished with value: 4.024548538315959 and parameters: {'n_estimators': 199, 'max_depth': 10}. Best is trial 1 with value: 4.024548538315959.
[I 2024-03-16 17:45:00,463] Trial 2 finished with value: 4.066956291305843 and parameters: {'n_estimators': 140, 'max_depth': 14}. Best is trial 1 with value: 4.024548538315959.
[I 2024-03-16 17:45:01,053] Trial 3 finished with value: 4.0354653278975885 and parameters: {'n_estimators': 114, 'max_depth': 17}. Best is trial 1 with value: 4.024548538315959.
[I 2024-03-16 17:45:01,766] Trial 4 finished with value: 4.0380897562533455 and parameters: {'n_estimators': 147, 'max_depth': 12}. Best is trial 1 with value: 4.02454853831

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,1.0,2.807645,0.666115,2.585546,0.120241,1.307433,0.938987,1.059462,0.051413,"{'n_estimators': 186, 'max_depth': 9}"


### KNeighborsRegressor

In [32]:
# Modelloptimalisering
knr_resultater = trening_knr_dag(trenings_andel,
                                trening_markert,
                                trening_umarkert,
                                trening,
                                X_test,
                                y_test,
                                splitt = splitt,
                                n_runder = n_runder)

# Lagring av resultater
knr_resultater_dag.loc[len(knr_resultater_dag)] = knr_resultater

# Data med resultater fra treningsandeler
knr_resultater_dag

[I 2024-03-16 17:46:25,738] A new study created in memory with name: no-name-99646a51-22d3-437d-8051-d000f67ca759
[I 2024-03-16 17:46:26,020] Trial 0 finished with value: 5.11595784541109 and parameters: {'n_neighbors': 12, 'p': 1}. Best is trial 0 with value: 5.11595784541109.
[I 2024-03-16 17:46:26,318] Trial 1 finished with value: 4.862792894066385 and parameters: {'n_neighbors': 9, 'p': 7}. Best is trial 1 with value: 4.862792894066385.
[I 2024-03-16 17:46:26,602] Trial 2 finished with value: 5.375085725449036 and parameters: {'n_neighbors': 17, 'p': 7}. Best is trial 1 with value: 4.862792894066385.
[I 2024-03-16 17:46:26,879] Trial 3 finished with value: 6.155409805237482 and parameters: {'n_neighbors': 1, 'p': 3}. Best is trial 1 with value: 4.862792894066385.
[I 2024-03-16 17:46:27,169] Trial 4 finished with value: 4.820192145362142 and parameters: {'n_neighbors': 9, 'p': 14}. Best is trial 4 with value: 4.820192145362142.
[I 2024-03-16 17:46:27,447] Trial 5 finished with value

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,1.0,4.138079,0.274714,3.6085,0.172763,3.584492,0.541397,2.870548,0.141477,"{'n_neighbors': 10, 'p': 2}"


### SupoortVectorRegressor

In [33]:
# Modelloptimalisering
svr_resultater = trening_svr_dag(trenings_andel,
                                trening_markert,
                                trening_umarkert,
                                trening,
                                X_test,
                                y_test,
                                splitt = splitt,
                                n_runder = n_runder)

# Lagring av resultater
svr_resultater_dag.loc[len(svr_resultater_dag)] = svr_resultater

# Data med resultater fra treningsandeler
svr_resultater_dag

[I 2024-03-16 17:46:58,587] A new study created in memory with name: no-name-bdfee835-c2e2-47ae-b013-f7acad0da455
[I 2024-03-16 17:46:59,034] Trial 0 finished with value: 8.404763838208545 and parameters: {'kernel': 'linear', 'C': 5, 'degree': 13, 'gamma': 'auto'}. Best is trial 0 with value: 8.404763838208545.
[I 2024-03-16 17:46:59,458] Trial 1 finished with value: 118498626627489.97 and parameters: {'kernel': 'poly', 'C': 21, 'degree': 13, 'gamma': 'auto'}. Best is trial 0 with value: 8.404763838208545.
[I 2024-03-16 17:46:59,995] Trial 2 finished with value: 8.357935352634376 and parameters: {'kernel': 'linear', 'C': 67, 'degree': 20, 'gamma': 'auto'}. Best is trial 2 with value: 8.357935352634376.
[I 2024-03-16 17:47:00,351] Trial 3 finished with value: 5.168309096964711 and parameters: {'kernel': 'rbf', 'C': 41, 'degree': 14, 'gamma': 'scale'}. Best is trial 3 with value: 5.168309096964711.
[I 2024-03-16 17:47:00,633] Trial 4 finished with value: 5.167677543121535 and parameters:

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,1.0,3.306285,0.536988,2.922926,0.131266,2.358248,0.8015,1.530043,0.076212,"{'kernel': 'rbf', 'C': 8, 'degree': 13, 'gamma..."


## Alternativ 2: Flere treningssandeler

In [34]:
# Dataframes for lagring av resultater
rfr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                          collagen_data,
                                                          test_andel=test_andel,
                                                          trening_krav=True)

# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [0.8, 0.85, 0.9, 0.95, 1.0]

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 300

### RandomForestRegressor

In [35]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening



  # Modelloptimalisering
  rfr_resultater = trening_rfr_dag(trenings_andel,
                                treningssett_markert,
                                treningssett_umarkert,
                                treningssett,
                                X_test,
                                y_test,
                                splitt = splitt,
                                n_runder = n_runder)

  # Lagring av resultater
  rfr_resultater_dag.loc[len(rfr_resultater_dag)] = rfr_resultater

# Data med resultater fra treningsandeler
rfr_resultater_dag

[I 2024-03-16 17:47:31,862] A new study created in memory with name: no-name-98581a3f-63c4-4970-b21d-e48b799b1d26
[I 2024-03-16 17:47:32,459] Trial 0 finished with value: 3.6871166523613716 and parameters: {'n_estimators': 104, 'max_depth': 14}. Best is trial 0 with value: 3.6871166523613716.
[I 2024-03-16 17:47:33,272] Trial 1 finished with value: 3.8058538314594568 and parameters: {'n_estimators': 198, 'max_depth': 8}. Best is trial 0 with value: 3.6871166523613716.
[I 2024-03-16 17:47:33,943] Trial 2 finished with value: 3.7520769025771394 and parameters: {'n_estimators': 146, 'max_depth': 17}. Best is trial 0 with value: 3.6871166523613716.
[I 2024-03-16 17:47:34,735] Trial 3 finished with value: 3.8183113571367695 and parameters: {'n_estimators': 192, 'max_depth': 6}. Best is trial 0 with value: 3.6871166523613716.
[I 2024-03-16 17:47:35,558] Trial 4 finished with value: 3.794753458488138 and parameters: {'n_estimators': 195, 'max_depth': 9}. Best is trial 0 with value: 3.68711665

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.80,3.240637,0.555192,2.964175,0.135855,1.411288,0.928515,1.085008,0.054372,"{'n_estimators': 106, 'max_depth': 14}"
1,0.85,2.988089,0.621820,2.762465,0.126942,1.374129,0.935863,1.107122,0.053697,"{'n_estimators': 196, 'max_depth': 19}"
2,0.90,2.837612,0.658950,2.527957,0.116361,1.908332,0.872325,1.594572,0.077150,"{'n_estimators': 199, 'max_depth': 4}"
3,0.95,2.780309,0.672586,2.537802,0.115792,1.367480,0.934951,1.113007,0.054416,"{'n_estimators': 144, 'max_depth': 8}"
4,1.00,2.807645,0.666115,2.585546,0.120241,1.307433,0.938987,1.059462,0.051413,"{'n_estimators': 186, 'max_depth': 9}"


### KNeighborRegressor

In [36]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  knr_resultater = trening_knr_dag(trenings_andel,
                                treningssett_markert,
                                treningssett_umarkert,
                                treningssett,
                                X_test,
                                y_test,
                                splitt = splitt,
                                n_runder = n_runder)

  # Lagring av resultater
  knr_resultater_dag.loc[len(knr_resultater_dag)] = knr_resultater

# Data med resultater fra treningsandeler
knr_resultater_dag

[I 2024-03-16 18:08:24,554] A new study created in memory with name: no-name-31ce26d4-6669-4afc-8b5d-38053e3ccbb8
[I 2024-03-16 18:08:24,857] Trial 0 finished with value: 4.545759925944374 and parameters: {'n_neighbors': 10, 'p': 4}. Best is trial 0 with value: 4.545759925944374.
[I 2024-03-16 18:08:25,159] Trial 1 finished with value: 4.254726600066334 and parameters: {'n_neighbors': 3, 'p': 20}. Best is trial 1 with value: 4.254726600066334.
[I 2024-03-16 18:08:25,451] Trial 2 finished with value: 4.254726600066334 and parameters: {'n_neighbors': 3, 'p': 15}. Best is trial 1 with value: 4.254726600066334.
[I 2024-03-16 18:08:25,744] Trial 3 finished with value: 4.664236706942539 and parameters: {'n_neighbors': 2, 'p': 5}. Best is trial 1 with value: 4.254726600066334.
[I 2024-03-16 18:08:26,039] Trial 4 finished with value: 5.0885718323968065 and parameters: {'n_neighbors': 16, 'p': 15}. Best is trial 1 with value: 4.254726600066334.
[I 2024-03-16 18:08:26,391] Trial 5 finished with 

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.80,4.189834,0.256458,3.760625,0.174729,3.503118,0.559552,2.787672,0.138287,"{'n_neighbors': 5, 'p': 12}"
1,0.85,4.471653,0.153069,3.843304,0.185947,3.520300,0.579070,2.845065,0.142394,"{'n_neighbors': 7, 'p': 2}"
2,0.90,4.085362,0.293076,3.465000,0.166463,3.503951,0.569557,2.852107,0.140877,"{'n_neighbors': 8, 'p': 2}"
3,0.95,4.133218,0.276417,3.544219,0.169316,3.485391,0.577426,2.830229,0.139531,"{'n_neighbors': 8, 'p': 2}"
4,1.00,4.148942,0.270901,3.572222,0.171822,3.570374,0.545003,2.881522,0.142378,"{'n_neighbors': 9, 'p': 2}"


### SupportVectorRegressor

In [37]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  svr_resultater = trening_svr_dag(trenings_andel,
                                treningssett_markert,
                                treningssett_umarkert,
                                treningssett,
                                X_test,
                                y_test,
                                splitt = splitt,
                                n_runder = n_runder)

  # Lagring av resultater
  svr_resultater_dag.loc[len(svr_resultater_dag)] = svr_resultater

# Data med resultater fra treningsandeler
svr_resultater_dag

[I 2024-03-16 18:16:57,154] A new study created in memory with name: no-name-603b8cf8-aae9-48d6-847d-9989be8cfabf
[I 2024-03-16 18:16:57,468] Trial 0 finished with value: 9.144929306106924 and parameters: {'kernel': 'linear', 'C': 82, 'degree': 20, 'gamma': 'scale'}. Best is trial 0 with value: 9.144929306106924.
[I 2024-03-16 18:16:57,762] Trial 1 finished with value: 4.112044998389486 and parameters: {'kernel': 'rbf', 'C': 20, 'degree': 9, 'gamma': 'scale'}. Best is trial 1 with value: 4.112044998389486.
[I 2024-03-16 18:16:58,115] Trial 2 finished with value: 9.129104365219352 and parameters: {'kernel': 'linear', 'C': 96, 'degree': 2, 'gamma': 'auto'}. Best is trial 1 with value: 4.112044998389486.
[I 2024-03-16 18:16:58,418] Trial 3 finished with value: 8.553354113562857e+19 and parameters: {'kernel': 'poly', 'C': 12, 'degree': 20, 'gamma': 'auto'}. Best is trial 1 with value: 4.112044998389486.
[I 2024-03-16 18:16:58,694] Trial 4 finished with value: 4.160012503237768 and paramete

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.80,3.129988,0.585049,2.598651,0.114544,1.586762,0.909633,0.770478,0.038266,"{'kernel': 'rbf', 'C': 40, 'degree': 13, 'gamm..."
1,0.85,3.782169,0.394110,3.339456,0.148630,2.326215,0.816198,1.406527,0.067051,"{'kernel': 'rbf', 'C': 8, 'degree': 9, 'gamma'..."
2,0.90,3.574958,0.458681,3.179117,0.142143,2.542359,0.773393,1.648381,0.082789,"{'kernel': 'rbf', 'C': 6, 'degree': 7, 'gamma'..."
3,0.95,3.573289,0.459186,3.190412,0.142407,2.448031,0.791535,1.576088,0.078283,"{'kernel': 'rbf', 'C': 7, 'degree': 6, 'gamma'..."
4,1.00,3.306285,0.536988,2.922926,0.131266,2.358248,0.801500,1.530043,0.076212,"{'kernel': 'rbf', 'C': 8, 'degree': 20, 'gamma..."


## Designproduksjon

## Alternativ 1: Flere treningsandeler

In [38]:
# Dataframes for lagring av resultater
rfr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)


# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, _, trening_umarkert,\
_, trening = trening_testsett_oppdeling_enzym(rå_data,
                                              collagen_data_design,
                                              test_andel=test_andel)

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [0.9, 1.0]

# Parametervalg for kryssvalidering
splitt = 2
gjentagelser = 10

# Antall runder til optimalisering
n_runder = 300

### RandomForestRegressor

In [39]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening


  # Modelloptimalisering
  rfr_resultater = trening_rfr_enzym(treningssett_markert,
                                     enzymtyper_treningssett_markert,
                                     treningssett_umarkert,
                                     treningssett,
                                     X_test,
                                     y_test,
                                     splitt = splitt,
                                     gjentagelser = gjentagelser,
                                     n_runder = n_runder,
                                     trenings_andel=trenings_andel)

  # Lagring av resultater
  rfr_resultater_enzym.loc[len(rfr_resultater_enzym)] = rfr_resultater

# Data med resultater fra treningsandeler
rfr_resultater_enzym

[I 2024-03-16 18:25:36,254] A new study created in memory with name: no-name-8db9ad6d-cdce-4b8c-8137-d0fa532bfe0d
[I 2024-03-16 18:25:41,982] Trial 0 finished with value: 3.853457158637499 and parameters: {'n_estimators': 168, 'max_depth': 15}. Best is trial 0 with value: 3.853457158637499.
[I 2024-03-16 18:25:45,610] Trial 1 finished with value: 3.8489134300702244 and parameters: {'n_estimators': 141, 'max_depth': 6}. Best is trial 1 with value: 3.8489134300702244.
[I 2024-03-16 18:25:49,754] Trial 2 finished with value: 3.8520905772196374 and parameters: {'n_estimators': 162, 'max_depth': 10}. Best is trial 1 with value: 3.8489134300702244.
[I 2024-03-16 18:25:54,587] Trial 3 finished with value: 4.124518290098983 and parameters: {'n_estimators': 136, 'max_depth': 1}. Best is trial 1 with value: 3.8489134300702244.
[I 2024-03-16 18:25:57,881] Trial 4 finished with value: 3.8557432063137522 and parameters: {'n_estimators': 127, 'max_depth': 10}. Best is trial 1 with value: 3.848913430

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.9,3.585513,0.377753,2.987888,0.142826,1.276665,0.938262,0.979782,0.045327,"{'n_estimators': 184, 'max_depth': 9}"
1,1.0,3.083196,0.539889,2.605247,0.124014,1.270260,0.936444,0.971876,0.044720,"{'n_estimators': 113, 'max_depth': 10}"


### KNeighborRegressor

In [40]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  knr_resultater = trening_knr_enzym(treningssett_markert,
                                     enzymtyper_treningssett_markert,
                                     treningssett_umarkert,
                                     treningssett,
                                     X_test,
                                     y_test,
                                     splitt=splitt,
                                     gjentagelser=gjentagelser,
                                     n_runder=n_runder,
                                     trenings_andel=trenings_andel)


  # Lagring av resultater
  knr_resultater_enzym.loc[len(knr_resultater_enzym)] = knr_resultater

# Data med resultater fra treningsandeler
knr_resultater_enzym

[I 2024-03-16 19:09:16,527] A new study created in memory with name: no-name-c29b5634-06ad-467d-8041-33ad19e131e6
[I 2024-03-16 19:09:16,683] Trial 0 finished with value: 5.479581228569286 and parameters: {'n_neighbors': 1, 'p': 9}. Best is trial 0 with value: 5.479581228569286.
[I 2024-03-16 19:09:16,837] Trial 1 finished with value: 4.82388729812585 and parameters: {'n_neighbors': 2, 'p': 10}. Best is trial 1 with value: 4.82388729812585.
[I 2024-03-16 19:09:17,009] Trial 2 finished with value: 4.714606493165923 and parameters: {'n_neighbors': 10, 'p': 7}. Best is trial 2 with value: 4.714606493165923.
[I 2024-03-16 19:09:17,183] Trial 3 finished with value: 4.543452716515042 and parameters: {'n_neighbors': 6, 'p': 7}. Best is trial 3 with value: 4.543452716515042.
[I 2024-03-16 19:09:17,353] Trial 4 finished with value: 5.398550037413062 and parameters: {'n_neighbors': 1, 'p': 2}. Best is trial 3 with value: 4.543452716515042.
[I 2024-03-16 19:09:17,522] Trial 5 finished with value:

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.9,4.440457,0.045633,3.540556,0.180371,3.480766,0.541068,2.966992,0.13788,"{'n_neighbors': 6, 'p': 2}"
1,1.0,4.685372,-0.062547,3.880000,0.194406,3.628110,0.481520,3.023571,0.14142,"{'n_neighbors': 7, 'p': 2}"


### SupportVectorRegressor

In [41]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  svr_resultater = trening_svr_enzym(treningssett_markert,
                                     enzymtyper_treningssett_markert,
                                     treningssett_umarkert,
                                     treningssett,
                                     X_test,
                                     y_test,
                                     splitt=splitt,
                                     gjentagelser=gjentagelser,
                                     n_runder=n_runder,
                                     trenings_andel=trenings_andel)


  # Lagring av resultater
  svr_resultater_enzym.loc[len(svr_resultater_enzym)] = svr_resultater

# Data med resultater fra treningsandeler
svr_resultater_enzym

[I 2024-03-16 19:11:17,431] A new study created in memory with name: no-name-19535b05-19f2-432d-bbd3-2b843b47a374
[I 2024-03-16 19:11:17,575] Trial 0 finished with value: 4.664648338066827 and parameters: {'kernel': 'rbf', 'C': 59, 'degree': 16, 'gamma': 'scale'}. Best is trial 0 with value: 4.664648338066827.
[I 2024-03-16 19:11:17,719] Trial 1 finished with value: 4.633961952287966 and parameters: {'kernel': 'rbf', 'C': 37, 'degree': 1, 'gamma': 'scale'}. Best is trial 1 with value: 4.633961952287966.
[I 2024-03-16 19:11:17,899] Trial 2 finished with value: 5.039213946573363 and parameters: {'kernel': 'linear', 'C': 18, 'degree': 3, 'gamma': 'scale'}. Best is trial 1 with value: 4.633961952287966.
[I 2024-03-16 19:11:18,050] Trial 3 finished with value: 4.462644618705972 and parameters: {'kernel': 'linear', 'C': 1, 'degree': 19, 'gamma': 'auto'}. Best is trial 3 with value: 4.462644618705972.
[I 2024-03-16 19:11:18,193] Trial 4 finished with value: 33.84974397299622 and parameters: {

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.9,4.716829,-0.076863,4.035797,0.202231,2.987790,0.661858,1.939108,0.091029,"{'kernel': 'poly', 'C': 3, 'degree': 3, 'gamma..."
1,1.0,3.380275,0.446950,2.494900,0.117028,1.149519,0.947952,0.392586,0.017975,"{'kernel': 'rbf', 'C': 41, 'degree': 13, 'gamm..."


## Alternativ 2: Flere treningssandeler

In [42]:
# Dataframes for lagring av resultater
rfr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                          collagen_data_design,
                                                          test_andel=test_andel,
                                                          trening_krav=True)

# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [0.8, 0.85, 0.9, 0.95, 1.0]

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 300

### RandomForestRegressor

In [43]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening



  # Modelloptimalisering
  rfr_resultater = trening_rfr_dag(trenings_andel,
                                treningssett_markert,
                                treningssett_umarkert,
                                treningssett,
                                X_test,
                                y_test,
                                splitt = splitt,
                                n_runder = n_runder)

  # Lagring av resultater
  rfr_resultater_dag.loc[len(rfr_resultater_dag)] = rfr_resultater

# Data med resultater fra treningsandeler
rfr_resultater_dag

[I 2024-03-16 19:13:14,227] A new study created in memory with name: no-name-3455c5d6-b905-4eb1-be21-dff87b44d4e4
[I 2024-03-16 19:13:15,052] Trial 0 finished with value: 3.4092525799793827 and parameters: {'n_estimators': 143, 'max_depth': 7}. Best is trial 0 with value: 3.4092525799793827.
[I 2024-03-16 19:13:15,811] Trial 1 finished with value: 3.4080926894391723 and parameters: {'n_estimators': 124, 'max_depth': 6}. Best is trial 1 with value: 3.4080926894391723.
[I 2024-03-16 19:13:16,449] Trial 2 finished with value: 3.437836030010141 and parameters: {'n_estimators': 167, 'max_depth': 12}. Best is trial 1 with value: 3.4080926894391723.
[I 2024-03-16 19:13:16,989] Trial 3 finished with value: 3.440879313121415 and parameters: {'n_estimators': 152, 'max_depth': 12}. Best is trial 1 with value: 3.4080926894391723.
[I 2024-03-16 19:13:17,472] Trial 4 finished with value: 3.4359452515334854 and parameters: {'n_estimators': 117, 'max_depth': 10}. Best is trial 1 with value: 3.40809268

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.80,3.898371,0.260483,3.516586,0.163490,1.407272,0.918729,1.114421,0.051881,"{'n_estimators': 143, 'max_depth': 6}"
1,0.85,3.884573,0.265709,3.486206,0.164273,1.304891,0.938218,1.059605,0.047684,"{'n_estimators': 173, 'max_depth': 7}"
2,0.90,3.815829,0.291468,3.394575,0.161588,1.322629,0.934156,1.081873,0.048958,"{'n_estimators': 126, 'max_depth': 7}"
3,0.95,3.797735,0.298171,3.374506,0.159579,1.272572,0.939068,1.017181,0.045836,"{'n_estimators': 124, 'max_depth': 8}"
4,1.00,3.698362,0.334419,3.284340,0.156074,1.403876,0.922656,1.134816,0.050582,"{'n_estimators': 104, 'max_depth': 5}"


### KNeighborRegressor

In [44]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  knr_resultater = trening_knr_dag(trenings_andel,
                                treningssett_markert,
                                treningssett_umarkert,
                                treningssett,
                                X_test,
                                y_test,
                                splitt = splitt,
                                n_runder = n_runder)

  # Lagring av resultater
  knr_resultater_dag.loc[len(knr_resultater_dag)] = knr_resultater

# Data med resultater fra treningsandeler
knr_resultater_dag

[I 2024-03-16 19:27:58,603] A new study created in memory with name: no-name-13e8e306-8190-4a01-85fb-74f74cfc6312
[I 2024-03-16 19:27:58,765] Trial 0 finished with value: 4.966300729103519 and parameters: {'n_neighbors': 16, 'p': 2}. Best is trial 0 with value: 4.966300729103519.
[I 2024-03-16 19:27:58,933] Trial 1 finished with value: 5.295057709245981 and parameters: {'n_neighbors': 19, 'p': 17}. Best is trial 0 with value: 4.966300729103519.
[I 2024-03-16 19:27:59,101] Trial 2 finished with value: 5.148587912718754 and parameters: {'n_neighbors': 7, 'p': 19}. Best is trial 0 with value: 4.966300729103519.
[I 2024-03-16 19:27:59,272] Trial 3 finished with value: 5.029740471146316 and parameters: {'n_neighbors': 20, 'p': 2}. Best is trial 0 with value: 4.966300729103519.
[I 2024-03-16 19:27:59,446] Trial 4 finished with value: 4.826426399739524 and parameters: {'n_neighbors': 5, 'p': 14}. Best is trial 4 with value: 4.826426399739524.
[I 2024-03-16 19:27:59,614] Trial 5 finished with 

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.80,3.978273,0.229858,3.439667,0.168066,3.764746,0.418364,3.018661,0.143494,"{'n_neighbors': 9, 'p': 2}"
1,0.85,4.162716,0.156791,3.398200,0.171276,3.447169,0.568837,2.744286,0.133308,"{'n_neighbors': 5, 'p': 1}"
2,0.90,4.019183,0.213937,3.456800,0.169930,3.341847,0.579649,2.669956,0.127665,"{'n_neighbors': 5, 'p': 1}"
3,0.95,4.115689,0.175735,3.495417,0.172705,3.628671,0.504575,2.832210,0.138455,"{'n_neighbors': 12, 'p': 2}"
4,1.00,4.143885,0.164403,3.360143,0.169833,3.637243,0.480826,2.856101,0.137414,"{'n_neighbors': 7, 'p': 1}"


### SupportVectorRegressor

In [45]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  svr_resultater = trening_svr_dag(trenings_andel,
                                treningssett_markert,
                                treningssett_umarkert,
                                treningssett,
                                X_test,
                                y_test,
                                splitt = splitt,
                                n_runder = n_runder)

  # Lagring av resultater
  svr_resultater_dag.loc[len(svr_resultater_dag)] = svr_resultater

# Data med resultater fra treningsandeler
svr_resultater_dag

[I 2024-03-16 19:32:53,850] A new study created in memory with name: no-name-6351dc61-d803-4ef6-87d6-44fb216b008e
[I 2024-03-16 19:32:54,010] Trial 0 finished with value: 4.1705245439345 and parameters: {'kernel': 'rbf', 'C': 34, 'degree': 11, 'gamma': 'auto'}. Best is trial 0 with value: 4.1705245439345.
[I 2024-03-16 19:32:54,205] Trial 1 finished with value: 6.415189724374843 and parameters: {'kernel': 'poly', 'C': 84, 'degree': 8, 'gamma': 'scale'}. Best is trial 0 with value: 4.1705245439345.
[I 2024-03-16 19:32:54,382] Trial 2 finished with value: 4.381532867553005 and parameters: {'kernel': 'rbf', 'C': 80, 'degree': 17, 'gamma': 'scale'}. Best is trial 0 with value: 4.1705245439345.
[I 2024-03-16 19:32:54,562] Trial 3 finished with value: 4.230856743178787 and parameters: {'kernel': 'rbf', 'C': 17, 'degree': 19, 'gamma': 'auto'}. Best is trial 0 with value: 4.1705245439345.
[I 2024-03-16 19:32:54,727] Trial 4 finished with value: 4.168473598216683 and parameters: {'kernel': 'rbf

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.80,3.133732,0.522135,2.595441,0.130464,2.569426,0.729073,1.697532,0.079056,"{'kernel': 'linear', 'C': 10, 'degree': 6, 'ga..."
1,0.85,5.176480,-0.303920,4.530622,0.218701,3.052467,0.661921,1.831186,0.081874,"{'kernel': 'poly', 'C': 17, 'degree': 7, 'gamm..."
2,0.90,3.245395,0.487473,2.798845,0.136468,2.559670,0.753392,1.816163,0.083115,"{'kernel': 'linear', 'C': 3, 'degree': 1, 'gam..."
3,0.95,4.258792,0.117419,3.845872,0.181351,2.256645,0.808394,1.291675,0.053486,"{'kernel': 'poly', 'C': 19, 'degree': 5, 'gamm..."
4,1.00,3.035424,0.551647,2.583554,0.129350,2.645362,0.725376,1.922477,0.087359,"{'kernel': 'linear', 'C': 15, 'degree': 20, 'g..."


# NIR målinger ekskludert

In [49]:
# Ekskluderer NIR målinger fra datasettene
NIR_kolonner = ['NIRfat', 'NIRwater', 'NIRash']
collagen_data_uten_NIR = collagen_data.drop(columns=NIR_kolonner)
collagen_data_design_uten_NIR = collagen_data_design.drop(columns=NIR_kolonner)

## Hele datasett

## Alternativ 1: Flere treningsandeler

In [50]:
# Dataframes for lagring av resultater
rfr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)


# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, _, trening_umarkert,\
_, trening = trening_testsett_oppdeling_enzym(rå_data,
                                              collagen_data_uten_NIR,
                                              test_andel=test_andel)

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [1.0]

# Parametervalg for kryssvalidering
splitt = 2
gjentagelser = 10

# Antall runder til optimalisering
n_runder = 300

### RandomForestRegressor

In [51]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening


  # Modelloptimalisering
  rfr_resultater = trening_rfr_enzym(treningssett_markert,
                                     enzymtyper_treningssett_markert,
                                     treningssett_umarkert,
                                     treningssett,
                                     X_test,
                                     y_test,
                                     splitt = splitt,
                                     gjentagelser = gjentagelser,
                                     n_runder = n_runder,
                                     trenings_andel=trenings_andel)

  # Lagring av resultater
  rfr_resultater_enzym.loc[len(rfr_resultater_enzym)] = rfr_resultater

# Data med resultater fra treningsandeler
rfr_resultater_enzym

[I 2024-03-16 19:46:04,924] A new study created in memory with name: no-name-6e416b68-0f6e-45fc-b60f-a7df551892c1
[I 2024-03-16 19:46:10,272] Trial 0 finished with value: 5.194624527115271 and parameters: {'n_estimators': 150, 'max_depth': 6}. Best is trial 0 with value: 5.194624527115271.
[I 2024-03-16 19:46:13,331] Trial 1 finished with value: 5.2051906103720045 and parameters: {'n_estimators': 115, 'max_depth': 12}. Best is trial 0 with value: 5.194624527115271.
[I 2024-03-16 19:46:16,091] Trial 2 finished with value: 5.215701912961443 and parameters: {'n_estimators': 103, 'max_depth': 11}. Best is trial 0 with value: 5.194624527115271.
[I 2024-03-16 19:46:21,241] Trial 3 finished with value: 5.175800143908827 and parameters: {'n_estimators': 158, 'max_depth': 5}. Best is trial 3 with value: 5.175800143908827.
[I 2024-03-16 19:46:24,752] Trial 4 finished with value: 5.2070690824126995 and parameters: {'n_estimators': 122, 'max_depth': 19}. Best is trial 3 with value: 5.1758001439088

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,1.0,5.36574,0.171515,4.218444,0.227061,4.311776,0.266314,3.593041,0.172687,"{'n_estimators': 113, 'max_depth': 1}"


### KNeighborRegressor

In [52]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  knr_resultater = trening_knr_enzym(treningssett_markert,
                                     enzymtyper_treningssett_markert,
                                     treningssett_umarkert,
                                     treningssett,
                                     X_test,
                                     y_test,
                                     splitt=splitt,
                                     gjentagelser=gjentagelser,
                                     n_runder=n_runder,
                                     trenings_andel=trenings_andel)


  # Lagring av resultater
  knr_resultater_enzym.loc[len(knr_resultater_enzym)] = knr_resultater

# Data med resultater fra treningsandeler
knr_resultater_enzym

[I 2024-03-16 20:05:49,427] A new study created in memory with name: no-name-06d17bdc-3289-47a6-96c7-b9ff8c4dd6bf
[I 2024-03-16 20:05:49,587] Trial 0 finished with value: 4.9393867952685815 and parameters: {'n_neighbors': 6, 'p': 8}. Best is trial 0 with value: 4.9393867952685815.
[I 2024-03-16 20:05:49,738] Trial 1 finished with value: 4.929958087154551 and parameters: {'n_neighbors': 9, 'p': 9}. Best is trial 1 with value: 4.929958087154551.
[I 2024-03-16 20:05:49,886] Trial 2 finished with value: 5.116184648959022 and parameters: {'n_neighbors': 3, 'p': 8}. Best is trial 1 with value: 4.929958087154551.
[I 2024-03-16 20:05:50,047] Trial 3 finished with value: 4.957882569069092 and parameters: {'n_neighbors': 6, 'p': 3}. Best is trial 1 with value: 4.929958087154551.
[I 2024-03-16 20:05:50,221] Trial 4 finished with value: 5.4883484096861785 and parameters: {'n_neighbors': 2, 'p': 8}. Best is trial 1 with value: 4.929958087154551.
[I 2024-03-16 20:05:50,390] Trial 5 finished with val

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,1.0,4.871605,0.317081,4.145833,0.218425,4.057116,0.35042,3.287588,0.157875,"{'n_neighbors': 8, 'p': 10}"


### SupportVectorRegressor

In [53]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  svr_resultater = trening_svr_enzym(treningssett_markert,
                                     enzymtyper_treningssett_markert,
                                     treningssett_umarkert,
                                     treningssett,
                                     X_test,
                                     y_test,
                                     splitt=splitt,
                                     gjentagelser=gjentagelser,
                                     n_runder=n_runder,
                                     trenings_andel=trenings_andel)


  # Lagring av resultater
  svr_resultater_enzym.loc[len(svr_resultater_enzym)] = svr_resultater

# Data med resultater fra treningsandeler
svr_resultater_enzym

[I 2024-03-16 20:06:47,531] A new study created in memory with name: no-name-b5f9b697-e379-4be5-b577-49208f905ea2
[I 2024-03-16 20:06:47,693] Trial 0 finished with value: 9.610570181084379 and parameters: {'kernel': 'linear', 'C': 15, 'degree': 17, 'gamma': 'scale'}. Best is trial 0 with value: 9.610570181084379.
[I 2024-03-16 20:06:47,855] Trial 1 finished with value: 5.976294712342925 and parameters: {'kernel': 'rbf', 'C': 58, 'degree': 18, 'gamma': 'auto'}. Best is trial 1 with value: 5.976294712342925.
[I 2024-03-16 20:06:48,008] Trial 2 finished with value: 5.614089747779701 and parameters: {'kernel': 'rbf', 'C': 24, 'degree': 7, 'gamma': 'auto'}. Best is trial 2 with value: 5.614089747779701.
[I 2024-03-16 20:06:48,219] Trial 3 finished with value: 9.6036415180307 and parameters: {'kernel': 'linear', 'C': 41, 'degree': 8, 'gamma': 'auto'}. Best is trial 2 with value: 5.614089747779701.
[I 2024-03-16 20:06:48,367] Trial 4 finished with value: 8.680163413792616 and parameters: {'ke

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,1.0,5.449127,0.145565,4.358592,0.241331,4.331292,0.259657,3.291543,0.169058,"{'kernel': 'rbf', 'C': 1, 'degree': 10, 'gamma..."


## Alternativ 2: Flere treningssandeler

In [54]:
# Dataframes for lagring av resultater
rfr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                          collagen_data_uten_NIR,
                                                          test_andel=test_andel,
                                                          trening_krav=True)

# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [1.0]

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 300

### RandomForestRegressor

In [55]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening



  # Modelloptimalisering
  rfr_resultater = trening_rfr_dag(trenings_andel,
                                treningssett_markert,
                                treningssett_umarkert,
                                treningssett,
                                X_test,
                                y_test,
                                splitt = splitt,
                                n_runder = n_runder)

  # Lagring av resultater
  rfr_resultater_dag.loc[len(rfr_resultater_dag)] = rfr_resultater

# Data med resultater fra treningsandeler
rfr_resultater_dag

[I 2024-03-16 20:07:46,558] A new study created in memory with name: no-name-3b71c50c-5b6b-4b44-9884-ddef6ae40729
[I 2024-03-16 20:07:47,328] Trial 0 finished with value: 6.412763741643461 and parameters: {'n_estimators': 116, 'max_depth': 15}. Best is trial 0 with value: 6.412763741643461.
[I 2024-03-16 20:07:48,121] Trial 1 finished with value: 6.3520958564627 and parameters: {'n_estimators': 196, 'max_depth': 5}. Best is trial 1 with value: 6.3520958564627.
[I 2024-03-16 20:07:48,820] Trial 2 finished with value: 6.257246678616603 and parameters: {'n_estimators': 161, 'max_depth': 2}. Best is trial 2 with value: 6.257246678616603.
[I 2024-03-16 20:07:49,470] Trial 3 finished with value: 6.264013618057691 and parameters: {'n_estimators': 137, 'max_depth': 2}. Best is trial 2 with value: 6.257246678616603.
[I 2024-03-16 20:07:50,080] Trial 4 finished with value: 6.399464190117742 and parameters: {'n_estimators': 128, 'max_depth': 19}. Best is trial 2 with value: 6.257246678616603.
[I 

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,1.0,4.378691,0.187917,4.066642,0.195962,4.540054,0.264295,3.614085,0.179045,"{'n_estimators': 171, 'max_depth': 1}"


### KNeighborRegressor

In [56]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  knr_resultater = trening_knr_dag(trenings_andel,
                                treningssett_markert,
                                treningssett_umarkert,
                                treningssett,
                                X_test,
                                y_test,
                                splitt = splitt,
                                n_runder = n_runder)

  # Lagring av resultater
  knr_resultater_dag.loc[len(knr_resultater_dag)] = knr_resultater

# Data med resultater fra treningsandeler
knr_resultater_dag

[I 2024-03-16 20:11:36,810] A new study created in memory with name: no-name-2ab9bd21-ff38-43fb-9744-6d8a63607bb5
[I 2024-03-16 20:11:37,162] Trial 0 finished with value: 7.750101550281119 and parameters: {'n_neighbors': 1, 'p': 9}. Best is trial 0 with value: 7.750101550281119.
[I 2024-03-16 20:11:37,447] Trial 1 finished with value: 5.913257888720207 and parameters: {'n_neighbors': 10, 'p': 12}. Best is trial 1 with value: 5.913257888720207.
[I 2024-03-16 20:11:37,729] Trial 2 finished with value: 5.935716950872303 and parameters: {'n_neighbors': 13, 'p': 8}. Best is trial 1 with value: 5.913257888720207.
[I 2024-03-16 20:11:38,025] Trial 3 finished with value: 6.020589383808162 and parameters: {'n_neighbors': 4, 'p': 5}. Best is trial 1 with value: 5.913257888720207.
[I 2024-03-16 20:11:38,323] Trial 4 finished with value: 6.179541368645827 and parameters: {'n_neighbors': 20, 'p': 6}. Best is trial 1 with value: 5.913257888720207.
[I 2024-03-16 20:11:38,599] Trial 5 finished with va

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,1.0,4.910422,-0.021291,4.322917,0.209094,4.206112,0.368544,3.355753,0.165392,"{'n_neighbors': 9, 'p': 13}"


### SupportVectorRegressor

In [57]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  svr_resultater = trening_svr_dag(trenings_andel,
                                treningssett_markert,
                                treningssett_umarkert,
                                treningssett,
                                X_test,
                                y_test,
                                splitt = splitt,
                                n_runder = n_runder)

  # Lagring av resultater
  svr_resultater_dag.loc[len(svr_resultater_dag)] = svr_resultater

# Data med resultater fra treningsandeler
svr_resultater_dag

[I 2024-03-16 20:13:16,228] A new study created in memory with name: no-name-ee08b834-c663-4dad-83fe-8627aa23f292
[I 2024-03-16 20:13:16,517] Trial 0 finished with value: 6.699874220243111 and parameters: {'kernel': 'rbf', 'C': 99, 'degree': 12, 'gamma': 'scale'}. Best is trial 0 with value: 6.699874220243111.
[I 2024-03-16 20:13:16,795] Trial 1 finished with value: 5.858933149901979 and parameters: {'kernel': 'rbf', 'C': 3, 'degree': 7, 'gamma': 'auto'}. Best is trial 1 with value: 5.858933149901979.
[I 2024-03-16 20:13:17,115] Trial 2 finished with value: 6.729749645984671 and parameters: {'kernel': 'rbf', 'C': 81, 'degree': 11, 'gamma': 'scale'}. Best is trial 1 with value: 5.858933149901979.
[I 2024-03-16 20:13:17,404] Trial 3 finished with value: 10.541120805627159 and parameters: {'kernel': 'linear', 'C': 36, 'degree': 10, 'gamma': 'scale'}. Best is trial 1 with value: 5.858933149901979.
[I 2024-03-16 20:13:17,690] Trial 4 finished with value: 1.1995106294337649e+20 and parameter

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,1.0,4.683243,0.071022,4.184019,0.19799,4.188638,0.37378,2.831823,0.148279,"{'kernel': 'rbf', 'C': 6, 'degree': 12, 'gamma..."


## Designproduksjon

## Alternativ 1: Flere treningsandeler

In [58]:
# Dataframes for lagring av resultater
rfr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)


# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, _, trening_umarkert,\
_, trening = trening_testsett_oppdeling_enzym(rå_data,
                                              collagen_data_design_uten_NIR,
                                              test_andel=test_andel)

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [1.0]

# Parametervalg for kryssvalidering
splitt = 2
gjentagelser = 10

# Antall runder til optimalisering
n_runder = 300

### RandomForestRegressor

In [59]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening


  # Modelloptimalisering
  rfr_resultater = trening_rfr_enzym(treningssett_markert,
                                     enzymtyper_treningssett_markert,
                                     treningssett_umarkert,
                                     treningssett,
                                     X_test,
                                     y_test,
                                     splitt = splitt,
                                     gjentagelser = gjentagelser,
                                     n_runder = n_runder,
                                     trenings_andel=trenings_andel)

  # Lagring av resultater
  rfr_resultater_enzym.loc[len(rfr_resultater_enzym)] = rfr_resultater

# Data med resultater fra treningsandeler
rfr_resultater_enzym

[I 2024-03-16 20:14:55,856] A new study created in memory with name: no-name-c2adaaed-0a4f-433d-b4fe-3bfc8255c0c7
[I 2024-03-16 20:15:01,764] Trial 0 finished with value: 5.001685697733743 and parameters: {'n_estimators': 180, 'max_depth': 2}. Best is trial 0 with value: 5.001685697733743.
[I 2024-03-16 20:15:05,853] Trial 1 finished with value: 5.034729949906342 and parameters: {'n_estimators': 161, 'max_depth': 15}. Best is trial 0 with value: 5.001685697733743.
[I 2024-03-16 20:15:11,420] Trial 2 finished with value: 5.038885045962886 and parameters: {'n_estimators': 194, 'max_depth': 7}. Best is trial 0 with value: 5.001685697733743.
[I 2024-03-16 20:15:16,836] Trial 3 finished with value: 5.041278669221487 and parameters: {'n_estimators': 187, 'max_depth': 10}. Best is trial 0 with value: 5.001685697733743.
[I 2024-03-16 20:15:20,763] Trial 4 finished with value: 5.034442716780292 and parameters: {'n_estimators': 154, 'max_depth': 20}. Best is trial 0 with value: 5.001685697733743

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,1.0,4.221297,0.137514,3.271072,0.168103,3.417755,0.539899,2.837985,0.129953,"{'n_estimators': 156, 'max_depth': 2}"


### KNeighborRegressor

In [60]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  knr_resultater = trening_knr_enzym(treningssett_markert,
                                     enzymtyper_treningssett_markert,
                                     treningssett_umarkert,
                                     treningssett,
                                     X_test,
                                     y_test,
                                     splitt=splitt,
                                     gjentagelser=gjentagelser,
                                     n_runder=n_runder,
                                     trenings_andel=trenings_andel)


  # Lagring av resultater
  knr_resultater_enzym.loc[len(knr_resultater_enzym)] = knr_resultater

# Data med resultater fra treningsandeler
knr_resultater_enzym

[I 2024-03-16 20:36:43,477] A new study created in memory with name: no-name-460911eb-ee00-4baf-b21a-26651f0e6adc
[I 2024-03-16 20:36:43,640] Trial 0 finished with value: 5.044654081537183 and parameters: {'n_neighbors': 2, 'p': 4}. Best is trial 0 with value: 5.044654081537183.
[I 2024-03-16 20:36:43,787] Trial 1 finished with value: 4.836870124909837 and parameters: {'n_neighbors': 8, 'p': 7}. Best is trial 1 with value: 4.836870124909837.
[I 2024-03-16 20:36:43,937] Trial 2 finished with value: 4.86553685714582 and parameters: {'n_neighbors': 9, 'p': 2}. Best is trial 1 with value: 4.836870124909837.
[I 2024-03-16 20:36:44,100] Trial 3 finished with value: 4.9570726760266615 and parameters: {'n_neighbors': 9, 'p': 6}. Best is trial 1 with value: 4.836870124909837.
[I 2024-03-16 20:36:44,253] Trial 4 finished with value: 4.654412589465339 and parameters: {'n_neighbors': 5, 'p': 7}. Best is trial 4 with value: 4.654412589465339.
[I 2024-03-16 20:36:44,407] Trial 5 finished with value:

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,1.0,4.739588,-0.08728,3.902833,0.191131,3.7933,0.433232,3.136435,0.143175,"{'n_neighbors': 5, 'p': 7}"


### SupportVectorRegressor

In [61]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  svr_resultater = trening_svr_enzym(treningssett_markert,
                                     enzymtyper_treningssett_markert,
                                     treningssett_umarkert,
                                     treningssett,
                                     X_test,
                                     y_test,
                                     splitt=splitt,
                                     gjentagelser=gjentagelser,
                                     n_runder=n_runder,
                                     trenings_andel=trenings_andel)


  # Lagring av resultater
  svr_resultater_enzym.loc[len(svr_resultater_enzym)] = svr_resultater

# Data med resultater fra treningsandeler
svr_resultater_enzym

[I 2024-03-16 20:37:39,104] A new study created in memory with name: no-name-e5589039-591e-46d1-864f-a1d3ddeef85d
[I 2024-03-16 20:37:39,256] Trial 0 finished with value: 5.962293720369725 and parameters: {'kernel': 'rbf', 'C': 76, 'degree': 15, 'gamma': 'scale'}. Best is trial 0 with value: 5.962293720369725.
[I 2024-03-16 20:37:39,410] Trial 1 finished with value: 5.538209881360859 and parameters: {'kernel': 'rbf', 'C': 40, 'degree': 6, 'gamma': 'scale'}. Best is trial 1 with value: 5.538209881360859.
[I 2024-03-16 20:37:39,556] Trial 2 finished with value: 17.39321627456838 and parameters: {'kernel': 'poly', 'C': 5, 'degree': 12, 'gamma': 'auto'}. Best is trial 1 with value: 5.538209881360859.
[I 2024-03-16 20:37:39,717] Trial 3 finished with value: 5.343064513797309 and parameters: {'kernel': 'linear', 'C': 53, 'degree': 10, 'gamma': 'scale'}. Best is trial 3 with value: 5.343064513797309.
[I 2024-03-16 20:37:39,875] Trial 4 finished with value: 108.54562975422881 and parameters: {

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,1.0,4.899486,-0.161879,4.01613,0.202578,3.888056,0.404563,2.247805,0.123148,"{'kernel': 'rbf', 'C': 4, 'degree': 10, 'gamma..."


## Alternativ 2: Flere treningssandeler

In [62]:
# Dataframes for lagring av resultater
rfr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                          collagen_data_design_uten_NIR,
                                                          test_andel=test_andel,
                                                          trening_krav=True)

# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [1.0]

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 300

### RandomForestRegressor

In [63]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening



  # Modelloptimalisering
  rfr_resultater = trening_rfr_dag(trenings_andel,
                                treningssett_markert,
                                treningssett_umarkert,
                                treningssett,
                                X_test,
                                y_test,
                                splitt = splitt,
                                n_runder = n_runder)

  # Lagring av resultater
  rfr_resultater_dag.loc[len(rfr_resultater_dag)] = rfr_resultater

# Data med resultater fra treningsandeler
rfr_resultater_dag

[I 2024-03-16 20:38:34,849] A new study created in memory with name: no-name-ff0a82b7-b9d4-40b7-9291-2286d5545941
[I 2024-03-16 20:38:35,319] Trial 0 finished with value: 5.400684540645942 and parameters: {'n_estimators': 115, 'max_depth': 5}. Best is trial 0 with value: 5.400684540645942.
[I 2024-03-16 20:38:36,154] Trial 1 finished with value: 5.415802695691474 and parameters: {'n_estimators': 171, 'max_depth': 20}. Best is trial 0 with value: 5.400684540645942.
[I 2024-03-16 20:38:36,992] Trial 2 finished with value: 5.404667959718802 and parameters: {'n_estimators': 152, 'max_depth': 8}. Best is trial 0 with value: 5.400684540645942.
[I 2024-03-16 20:38:37,694] Trial 3 finished with value: 5.449428699705331 and parameters: {'n_estimators': 117, 'max_depth': 7}. Best is trial 0 with value: 5.400684540645942.
[I 2024-03-16 20:38:38,724] Trial 4 finished with value: 5.385954877570114 and parameters: {'n_estimators': 194, 'max_depth': 11}. Best is trial 4 with value: 5.385954877570114.

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,1.0,4.36931,0.071018,3.661448,0.18003,3.41364,0.542697,2.783891,0.130762,"{'n_estimators': 100, 'max_depth': 2}"


### KNeighborRegressor

In [64]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  knr_resultater = trening_knr_dag(trenings_andel,
                                treningssett_markert,
                                treningssett_umarkert,
                                treningssett,
                                X_test,
                                y_test,
                                splitt = splitt,
                                n_runder = n_runder)

  # Lagring av resultater
  knr_resultater_dag.loc[len(knr_resultater_dag)] = knr_resultater

# Data med resultater fra treningsandeler
knr_resultater_dag

[I 2024-03-16 20:41:19,093] A new study created in memory with name: no-name-a76eab3c-3e34-4814-a49d-9f8a9b807135
[I 2024-03-16 20:41:19,361] Trial 0 finished with value: 5.870567648335169 and parameters: {'n_neighbors': 15, 'p': 15}. Best is trial 0 with value: 5.870567648335169.
[I 2024-03-16 20:41:19,592] Trial 1 finished with value: 5.8724406683463215 and parameters: {'n_neighbors': 3, 'p': 15}. Best is trial 0 with value: 5.870567648335169.
[I 2024-03-16 20:41:19,815] Trial 2 finished with value: 6.277624130919058 and parameters: {'n_neighbors': 2, 'p': 10}. Best is trial 0 with value: 5.870567648335169.
[I 2024-03-16 20:41:20,039] Trial 3 finished with value: 5.939142401498861 and parameters: {'n_neighbors': 19, 'p': 11}. Best is trial 0 with value: 5.870567648335169.
[I 2024-03-16 20:41:20,259] Trial 4 finished with value: 5.631690437872191 and parameters: {'n_neighbors': 7, 'p': 16}. Best is trial 4 with value: 5.631690437872191.
[I 2024-03-16 20:41:20,469] Trial 5 finished wit

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,1.0,4.654345,-0.054142,4.315571,0.202235,3.872963,0.411352,3.013274,0.145074,"{'n_neighbors': 7, 'p': 13}"


### SupportVectorRegressor

In [65]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  svr_resultater = trening_svr_dag(trenings_andel,
                                treningssett_markert,
                                treningssett_umarkert,
                                treningssett,
                                X_test,
                                y_test,
                                splitt = splitt,
                                n_runder = n_runder)

  # Lagring av resultater
  svr_resultater_dag.loc[len(svr_resultater_dag)] = svr_resultater

# Data med resultater fra treningsandeler
svr_resultater_dag

[I 2024-03-16 20:42:16,987] A new study created in memory with name: no-name-5a934353-2cae-4543-ab7d-627ba6dec7ac
[I 2024-03-16 20:42:17,191] Trial 0 finished with value: 5.534428767889894 and parameters: {'kernel': 'linear', 'C': 81, 'degree': 7, 'gamma': 'scale'}. Best is trial 0 with value: 5.534428767889894.
[I 2024-03-16 20:42:17,352] Trial 1 finished with value: 6.476083100389486 and parameters: {'kernel': 'poly', 'C': 99, 'degree': 11, 'gamma': 'auto'}. Best is trial 0 with value: 5.534428767889894.
[I 2024-03-16 20:42:17,509] Trial 2 finished with value: 6.6158403488514335 and parameters: {'kernel': 'poly', 'C': 68, 'degree': 13, 'gamma': 'scale'}. Best is trial 0 with value: 5.534428767889894.
[I 2024-03-16 20:42:17,669] Trial 3 finished with value: 5.534362048003375 and parameters: {'kernel': 'linear', 'C': 24, 'degree': 5, 'gamma': 'auto'}. Best is trial 3 with value: 5.534362048003375.
[I 2024-03-16 20:42:17,826] Trial 4 finished with value: 6.227227637199661 and parameters

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,1.0,4.248247,0.121784,4.022834,0.184562,4.118898,0.33422,2.769892,0.142444,"{'kernel': 'linear', 'C': 1, 'degree': 16, 'ga..."
